In [1]:
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="kurtosistest only valid for n>=20")

# Load the datasets
alpha_results_2018_2021 = pd.read_excel('Alpha_Results_2018_2021.xlsx')
alpha_results_2021_2024 = pd.read_excel('Alpha_Results_2021_2024.xlsx')

# Combine both results for a comprehensive analysis
alpha_results = pd.concat([alpha_results_2018_2021, alpha_results_2021_2024], ignore_index=True)

# Descriptive statistics for alpha values
descriptive_stats = alpha_results['Alpha'].describe()
print("Descriptive Statistics for Alpha Values:")
print(descriptive_stats)

# Detecting outliers using IQR method
Q1 = alpha_results['Alpha'].quantile(0.25)
Q3 = alpha_results['Alpha'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify non-outliers with progress bar
non_outliers = alpha_results[(alpha_results['Alpha'] >= lower_bound) & (alpha_results['Alpha'] <= upper_bound)]
print(f"\nNumber of non-outliers: {non_outliers.shape[0]}")

# Save the non-outliers data for DID analysis
non_outliers.to_excel('Alpha_Results_No_Outliers.xlsx', index=False)
print("\nSaved non-outliers data to 'Alpha_Results_No_Outliers.xlsx'.")


Descriptive Statistics for Alpha Values:
count    21078.000000
mean        -0.143248
std          0.961219
min         -4.546442
25%         -0.721402
50%         -0.176779
75%          0.360570
max          5.864655
Name: Alpha, dtype: float64

Number of non-outliers: 20403

Saved non-outliers data to 'Alpha_Results_No_Outliers.xlsx'.


In [6]:
import pandas as pd

# Load the datasets
alpha_data = pd.read_excel('Alpha_Results_No_Outliers.xlsx')
merged_data = pd.read_excel('Merged_Funds_Factors_Data.xlsx')

# Retrieve the Classification from merged_data based on ISIN
classification_data = merged_data[['ISIN', 'Classification']].drop_duplicates()

# Merge the classification data with the alpha data
alpha_data = pd.merge(alpha_data, classification_data, on='ISIN', how='left')

# Check if the merge was successful
print(alpha_data.head())

# Save the updated alpha data
alpha_data.to_excel('Alpha_With_Classification.xlsx', index=False)

print("Updated alpha data with classification saved to 'Alpha_With_Classification.xlsx'.")


           ISIN  Period     Alpha  R-Squared  \
0  LU0066794479    2018  0.123842   0.549134   
1  LU0066794479    2019 -1.451790   0.335561   
2  LU0066794479    2020 -1.058813   0.648356   
3  AT0000653670    2018 -0.098746   0.517367   
4  AT0000653670    2020 -0.569791   0.668382   

                                       Model Summary Classification  
0                              OLS Regression Res...      Article 8  
1                              OLS Regression Res...      Article 8  
2                              OLS Regression Res...      Article 8  
3                              OLS Regression Res...      Article 8  
4                              OLS Regression Res...      Article 8  
Updated alpha data with classification saved to 'Alpha_With_Classification.xlsx'.


In [8]:
import pandas as pd

# Load the updated alpha data with classification
alpha_data = pd.read_excel('Alpha_With_Classification.xlsx')

# Group by classification and calculate descriptive statistics for each article
descriptive_stats_by_classification = alpha_data.groupby('Classification')['Alpha'].describe()

# Display the descriptive statistics
print("Descriptive Statistics for Alpha Values by Classification:")
print(descriptive_stats_by_classification)

# Save the descriptive statistics to an Excel file
descriptive_stats_by_classification.to_excel('Descriptive_Stats_Alpha_By_Classification.xlsx')

print("Descriptive statistics by classification saved to 'Descriptive_Stats_Alpha_By_Classification.xlsx'.")


Descriptive Statistics for Alpha Values by Classification:
                  count      mean       std       min       25%       50%  \
Classification                                                              
Article 6        5679.0 -0.102864  0.813633 -2.343606 -0.626885 -0.126531   
Article 8       13040.0 -0.228368  0.800079 -2.343944 -0.767515 -0.224606   
Article 9        1684.0 -0.172931  0.765521 -2.334077 -0.654991 -0.136344   

                     75%       max  
Classification                      
Article 6       0.403627  1.976026  
Article 8       0.291778  1.980821  
Article 9       0.334045  1.980879  
Descriptive statistics by classification saved to 'Descriptive_Stats_Alpha_By_Classification.xlsx'.
